In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=1)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1437, 64), (360, 64), (1437,), (360,))

使用预创建的 Estimator 编写 TensorFlow 程序，大致需要执行下列几个步骤：<a href="https://www.tensorflow.org/guide/premade_estimators#overview_of_programming_with_estimators"><i class="fa fa-external-link-square" aria-hidden="true"></i></a>


1. 创建一个或多个输入函数。
2. 定义模型的特征列。
3. 实例化 Estimator，指定特征列和各种超参数。
4. 在 Estimator 对象上调用一个或多个方法，传递适当的输入函数作为数据的来源。

下面，我们就按 4 个步骤来完成。首先，输入到 Estimator 的训练、评估和预测的数据都必须要通过创建输入函数来完成。

输入函数是返回 `tf.data.Dataset` <a href="https://www.tensorflow.org/api_docs/python/tf/data/Dataset"><i class="fa fa-external-link-square" aria-hidden="true"></i></a> 对象的函数，此对象会输出下列含有两个元素的元组：

- `features` - Python 字典，其中：
    - 每个键都是特征的名称。
    - 每个值都是包含此特征所有值的数组。
- `label` - 包含每个样本的标签值的数组。

所以下面我们将原来的 NumPy 数组转换为 Pandas 提供的 DataFrame，这样就可以将方便将数据转换输入函数要求的 Python 字典类型。

In [13]:
X_train_ = pd.DataFrame(X_train, columns=[str(i) for i in range(64)])
X_test_ = pd.DataFrame(X_test, columns=[str(i) for i in range(64)])

y_train_ = pd.DataFrame(y_train, columns=['class'])
y_test_ = pd.DataFrame(y_test,columns=['class'])

In [17]:
dict(X_train_).keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63'])

我们直接开始定义数据输入函数 `input_fn`。`tf.data.Dataset` <a href="https://www.tensorflow.org/api_docs/python/tf/data/Dataset"><i class="fa fa-external-link-square" aria-hidden="true"></i></a> 对象是 TensorFlow 强烈推荐使用的数据管道。当数据为 `Dataset` 时，你就可以使用 TensorFlow 提供的一系列方法对数据进行变换，例如打乱采样，重复扩展，小批量输入等。

In [18]:
def input_fn(features, labels, batch_size):
    "数据输入函数"
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    dataset = dataset.repeat().batch(batch_size)
    
    return dataset

上面的代码中，`tf.data.Dataset.from_tensor_slices` 函数创建一个代表数组切片的 `tf.data.Dataset`。系统会在第一个维度内对该数组进行切片。

然后 `dataset` 执行了 `repeat` 重复序列操作，这样做得目的是保证后续能迭代更多次，否则当数据一遍轮完之后训练就终止了。`repeat()` 代表无限扩展，即到我们设定的迭代次数。`repeat(5)` 则表示重复序列 5 次，样本数据变为原来的 5 倍。接着，我们使用了 `batch` 每次从数据中取出 `batch_size` 的小批量进行迭代。

### 这样，创建数据输入函数就大功告成了。接下来，完成第二步：定义模型的特征列。

特征列是原始数据和 Estimator 之间的媒介，定义特征列就是告诉 Estimator 哪些是特征，每个特征的数据有什么特点。定义特征列并不是说指定几个字符串那样简单，我们需要利用 TensorFlow 提供的方法创建 Estimator 能够识别的特征列。

下面，我们将特征 DataFrame 的列名取出来，并使用 `tf.feature_column.numeric_column` <a href="https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column"><i class="fa fa-external-link-square" aria-hidden="true"></i></a> 将其转换为特征列。该方法即告诉 Estimator 特征是 Numeric 数值类型。更多类型的特征列可以参考官方文档 <a href="https://www.tensorflow.org/api_docs/python/tf/feature_column"><i class="fa fa-external-link-square" aria-hidden="true"></i></a>。


In [21]:
feature_columns = []
for key in X_train_.keys():
    feature_columns.append(
        tf.feature_column.numeric_column(key=key))
    
feature_columns[:3]

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

### 完成第 2 步之后，第 3 步是：实例化 Estimator，指定特征列和各种超参数。

这里使用 `tf.estimator.DNNClassifier`，需要传入特征列并定义隐含层神经元数量即目标值标签数量。

In [24]:
classifier = tf.estimator.DNNClassifier(
    # 特征列
    feature_columns = feature_columns,
    # 两个隐藏层
    hidden_units = [30,10],
  
    # 模型最终类别
    n_classes=10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/hx/j03xym054cg1ww68l_2ybzrc0000gn/T/tmp5s0y13qz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a39296310>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


紧接着，就可以在 Estimator 对象上调用一个或多个方法，传递适当的输入函数作为数据的来源。值得注意的是，这里将 `input_fn` 调用封装在 `lambda` 中以获取参数。

`steps` 参数告知方法在训练多步后停止训练。`steps` 和先前的 Epoch 不一样，此时相当于取出 `steps` 个 `batch_size` 的数据用于训练。而整个训练过程等价于 `steps * batch_size / 数据总量` 个 Epoch。<a href="https://stackoverflow.com/questions/38340311/what-is-the-difference-between-steps-and-epochs-in-tensorflow"><i class="fa fa-external-link-square" aria-hidden="true"></i></a> 所以，通过 `steps` 换算的 Epoch 可能不是整数，但这并不会影响到训练过程。

In [25]:
classifier.train(
    input_fn=lambda: input_fn(X_train_, y_train_, batch_size=64),
    steps=2000)

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/hx/j03xym054cg1ww68l_2ybzrc0000gn/T/tmp5s0y13qz/model.ckpt.
INFO:tensorflow:loss = 562.5636, step = 1
INFO:tensorflow:global_step/sec: 56.5817
INFO:tensorflow:loss = 59.606403, step = 101 (1.768 sec)
INFO:tensorflow:global_step/sec: 65.9308
INFO:tensorflow:loss = 48.60147, step = 201 (1.517 sec)
INFO:tensorflow:global_step/sec: 66.1768
INFO:tensorflow:loss = 24.108467, step = 301 (1.511 sec)
INFO:tensorflow:global_step/sec: 65.8426
INFO:tensorflow:loss = 17.322811, step = 401 (1.519 sec)
INFO:tensorflow:global_step/sec: 71.2922
INFO:tensorflow:loss = 24.540184, step = 501 (1.40

你可以看到上方训练执行的过程，权重会被自动存为检查点 `.ckpt` 文件。同时，后续的训练过程只有在 `loss` 更优时，检查点才会被覆盖。这样做的原因在于，后续的模型推理需要重载检查点权重，这样能保证存放的检查点性能状态最优。

In [26]:
def evaluate_input_fn(features, labels, batch_size):
    """评估数据输入函数"""
    # 将数据转换为 Dataset 对象
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    # 将仅处理成小批量
    dataset = dataset.batch(batch_size)
    return dataset

In [27]:
# 评估模型
eval_result = classifier.evaluate(
    input_fn=lambda: evaluate_input_fn(X_test_, y_test_, batch_size=64))

print('最终评估准确度：{:.3f}'.format(eval_result['accuracy']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-09T15:45:34Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/hx/j03xym054cg1ww68l_2ybzrc0000gn/T/tmp5s0y13qz/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-02-09-15:45:35
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9527778, average_loss = 0.18531853, global_step = 2000, loss = 11.119111
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /var/folders/hx/j03xym054cg1ww68l_2ybzrc0000gn/T/tmp5s0y13qz/model.ckpt-2000
最终评估准确度：0.953


### 官方推荐流程

1. 假设存在合适的预创建的 Estimator，使用它构建第一个模型并使用其结果确定基准。
2. 使用此预创建的 Estimator 构建和测试整体管道，包括数据的完整性和可靠性。
3. 如果存在其他合适的预创建的 Estimator，则运行实验来确定哪个预创建的 Estimator 效果最好。
4. 可以通过构建自定义 Estimator 进一步改进模型。

最终，我们就可以得到测试结果。除了大量的预创建 Estimator，你也可以根据自己的需要自定义 Estimator，这部分内容可以直接阅读 [<i class="fa fa-external-link-square" aria-hidden="true"> 官方文档相关章节</i>](https://www.tensorflow.org/guide/custom_estimators)。




### 神经网络搭建方法小结

- 利用 `tf.nn` 模块提供的各种神经网络组件和函数。
- 利用 `tf.keras` 模块提供的各种高阶神经网络层。
- 利用 `tf.estimator` 提供的高阶预创建或自定义封装模型。

大多情况下，使用这三种方法都可以实现我们的想法。不过，这里有一些使用经验供参考。

如果需要实现的网络自定义程度较高，有很多自己的想法且并没有合适的高阶 API 层供调用，那么首选肯定是 `tf.nn`。`tf.nn` 功能强大，但你需要自己管理图和会话，且大多数过程都需要利用 TensorFlow 一系列低阶 API 进行定义。

`tf.keras` 模块主要面向于实现包含标准化层的神经网络，例如后面会学习的经典卷积神经网络结构等。API 使用方便，简洁明了。

`tf.estimator` 本身在 `tf.keras`（原 `tf.layers`）之上构建而成，Keras 模型也可以通过创建 Estimator 进行训练 <a href="https://www.tensorflow.org/guide/estimators#creating_estimators_from_keras_models"><i class="fa fa-external-link-square" aria-hidden="true"></i></a>。Estimator 简化了在模型开发者之间共享实现的过程，其可以在本地主机上或分布式多服务器环境中运行基于 Estimator 的模型，而无需更改模型。但 Estimator 的使用可能需要对 TensorFlow 足够熟悉之后才能得心应手。